In [1]:
# pip install dotenv

In [1]:
from datetime import datetime
import os
import json
import requests
import hmac
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time
from config import API_KEY

from client import FtxClient

# load_dotenv()


ModuleNotFoundError: No module named 'ciso8601'

In [2]:
url = 'https://ftx.com/api/markets'

# get all market data as JSON
all_markets = requests.get(url).json()

# convert json to pd 
df = pd.DataFrame(all_markets['result'])
df.set_index('name', inplace=True)
df

,enabled,postOnly,priceIncrement,sizeIncrement,minProvideSize,last,bid,ask,price,type,...,highLeverageFeeExempt,largeOrderThreshold,change1h,change24h,changeBod,quoteVolume24h,volumeUsd24h,priceHigh24h,priceLow24h,tokenizedEquity
name,,,,,,,,,,,,,,,,,,,,,
1INCH-PERP,True,False,1.000000e-04,1.00000,1.00000,0.832300,0.832000,0.832300,0.832300,future,...,False,350.0,0.009583,0.058502,0.037005,6.588965e+06,6.588965e+06,0.852700,0.774400,NaN
1INCH-0930,True,False,1.000000e-04,1.00000,1.00000,0.837000,0.833800,0.839600,0.837000,future,...,False,350.0,0.017382,0.063262,0.040656,2.134454e+04,2.134454e+04,0.853300,0.777100,NaN
1INCH/USD,True,False,1.000000e-04,1.00000,1.00000,0.832400,0.832000,0.832400,0.832400,spot,...,True,350.0,0.011422,0.059303,0.037646,4.975747e+05,4.975747e+05,0.852200,0.774400,NaN
AAPL-0930,True,False,1.000000e-02,0.01000,0.01000,165.450000,165.590000,166.000000,165.590000,future,...,False,350.0,0.000000,0.000362,-0.000423,2.146618e+04,2.146618e+04,167.230000,164.540000,True
AAPL/USD,True,False,1.000000e-02,0.01000,0.01000,166.660000,165.930000,166.610000,166.610000,spot,...,True,350.0,0.000901,0.002769,0.001864,2.907041e+04,2.907041e+04,167.360000,164.740000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XTZBULL/USDT,True,False,1.000000e-08,1000.00000,1000.00000,0.000050,0.000050,0.000056,0.000050,spot,...,True,350.0,0.001205,0.069742,0.001205,4.422874e+03,4.423316e+03,0.000057,0.000045,NaN
XTZHALF/USD,True,False,5.000000e-01,0.00001,0.00001,6828.500000,6816.500000,6874.500000,6828.500000,spot,...,True,350.0,0.000000,0.006782,0.000000,0.000000e+00,0.000000e+00,6890.000000,6741.500000,NaN
XTZHEDGE/USD,True,False,5.000000e-03,0.00100,0.00100,60.105000,59.790000,60.685000,60.105000,spot,...,True,350.0,0.000000,-0.024269,0.000000,9.995462e+01,9.995462e+01,62.360000,59.215000,NaN


In [3]:
#get BTCUSD spot price

base_currency = 'BTC'
quote_currency = 'USD'

#construct request url
request_url = f"{url}/{base_currency}/{quote_currency}"

btcusd_df = pd.DataFrame(requests.get(request_url).json())
btcusd_df['result']

ask                             22675.0
baseCurrency                        BTC
bid                             22674.0
change1h                       0.001236
change24h                     -0.020773
changeBod                      0.002343
enabled                            True
futureType                         None
highLeverageFeeExempt              True
isEtfMarket                       False
largeOrderThreshold              3000.0
last                            22674.0
minProvideSize                   0.0001
name                            BTC/USD
postOnly                          False
price                           22674.0
priceHigh24h                    23227.0
priceIncrement                      1.0
priceLow24h                     22401.0
quoteCurrency                       USD
quoteVolume24h           503603727.7584
restricted                        False
sizeIncrement                    0.0001
type                               spot
underlying                         None


In [8]:
#1 day = 60*60*24 seconds
Hourly = str(60*60)

start_date = datetime(2022, 1, 1).timestamp()

#grab historical data as JSON
historical = requests.get(
    f'{request_url}/candles?resolution={Hourly}&start_time={start_date}'
).json()

#Convert JSON to pd df
df = pd.DataFrame(historical['result'])

#Convert time to date
df['date'] = pd.to_datetime(
    df['time']/1000, unit='s', origin='unix'
)

#remove unnecessary columns
df.drop(['startTime', 'time'], axis=1, inplace=True)

df

,open,high,low,close,volume,date
0,29696.0,29742.0,29267.0,29445.0,5.941973e+07,2022-06-03 14:00:00
1,29445.0,29545.0,29311.0,29471.0,3.882343e+07,2022-06-03 15:00:00
2,29471.0,29535.0,29375.0,29513.0,2.871500e+07,2022-06-03 16:00:00
3,29513.0,29647.0,29403.0,29472.0,4.203222e+07,2022-06-03 17:00:00
4,29472.0,29529.0,29432.0,29523.0,1.239665e+07,2022-06-03 18:00:00
...,...,...,...,...,...,...
1495,22512.0,22569.0,22484.0,22494.0,5.424622e+06,2022-08-04 21:00:00
1496,22494.0,22644.0,22449.0,22622.0,3.330921e+07,2022-08-04 22:00:00
1497,22622.0,22669.0,22571.0,22622.0,2.333638e+07,2022-08-04 23:00:00
1498,22622.0,22680.0,22591.0,22628.0,1.136323e+07,2022-08-05 00:00:00


In [9]:
# create a graph object
fig = go.Figure()

#set up layout
fig.update_layout(
    title={
        'text' : "BTCUSD",
        "x" : 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

#add candletstick
fig.add_trace(
    go.Candlestick(
        x=df['date'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    )
)

fig.show()